<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Baseline_model_with_acc_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 27.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=593146b27fc237bd745b9dc3b0cee050304bc4036baeb2f7e7153bff95f228f0
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc2

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc2) to /tmp/pip-req-build-igaboef6
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-igaboef6
  Running command git checkout -q 8f0d4550417af1676143499c688434065eb807c9
  Resolved https://github.com/keras-team/keras-tuner.git to commit 8f0d4550417af1676143499c688434065eb807c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.0 MB/s eta 0:00:00


# Importing Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda


In [ ]:
cBio = pd.read_csv('/content/drive/MyDrive/gsoc data/Acc_tcga_2018/cBio.csv')

In [ ]:
cBio = cBio.set_index('Sample Identifier')
cBio.index.name = None
cBio.head(2)

,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,ATG4A,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,385.687,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.55,363.112,1.9212,0.0
TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,547.980,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.32,592.672,0.0000,0.0


In [ ]:
#defining features and labels

y = np.array(cBio['Overall Survival (Months)'], dtype=float)
X = cBio.drop('Overall Survival (Months)', axis=1)

In [ ]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [ ]:
df = df.drop(columns='InteractionType')

In [ ]:
df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


# Selecting Common Nodes From Both Data

In [ ]:
all_nodes = sorted(list(set(df['Source']) | set(df['Target'])))
len(all_nodes)

12324

In [ ]:
len(set(all_nodes) & set(X.columns))

9614

In [ ]:
used_nodes = sorted(list(set(X.columns) & set(all_nodes)))
len(used_nodes)

9614

In [ ]:
X = X.loc[:, used_nodes]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAK1,AANAT,AARS,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,0.0000,121.0370,868.876,1.4409,3520.65,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,1.2079,31.0025,1627.030,0.0000,5470.94,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [ ]:
# Extract the values from the 'Source' column of the DataFrame
source_values = df['Source'].values

# Create a boolean mask indicating whether each element in 'source_values' is present in 'used_nodes'
mask = np.isin(source_values, used_nodes)

df = df.loc[mask]

# Update 'source_values' with the values from the 'Target' column of the filtered DataFrame
source_values = df['Target'].values

# Create a new boolean mask based on the updated 'source_values' array
mask = np.isin(source_values, used_nodes)


df = df.loc[mask]

# Reset the index of the filtered DataFrame, dropping the old index
df = df.reset_index(drop=True)

In [ ]:
len(set(df['Source']) | set(df['Target']))

9288

In [ ]:

# Select the columns from DataFrame 'X' that correspond to unique values
columns_to_select = sorted(set(df['Source']).union(set(df['Target'])))
X = X[columns_to_select]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


# Splitting the Data into Train and Test Splits

In [ ]:
# Set the fixed index for splitting
split_index = int(0.8 * X.shape[0])

# Split the data based on the fixed index
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [ ]:
X_train.shape, X_test.shape

((62, 9288), (16, 9288))

In [ ]:
# X_train = X_train.values
# X_test = X_test.values

In [ ]:
X_train

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0000,10373.70,0.0000,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.120,256.004,0.0000,253.602,183.9580,146.9740,596.062
TCGA-OR-A5J2-01,0.0000,9844.91,0.0000,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.480,104.684,0.4026,270.165,264.1250,438.4640,801.637
TCGA-OR-A5J3-01,0.5925,7201.84,2.3700,1259.67,5182.050,39.1053,1158.350,0.0000,4042.66,291.512,...,0.0,333.5090,0.0,864.465,193.749,1.1850,297.437,90.6532,190.1940,437.269
TCGA-OR-A5J5-01,1.5492,2939.04,0.7746,2993.80,967.467,98.3734,646.785,1.5492,2738.96,533.687,...,0.0,358.6060,0.0,1803.250,472.502,5.4222,146.398,260.2630,840.4340,512.781
TCGA-OR-A5J6-01,0.0000,9586.96,0.5589,1186.45,1479.290,19.5600,1238.990,0.0000,2996.59,414.102,...,0.0,143.6260,0.0,1110.450,282.223,0.0000,384.493,64.2685,63.1508,475.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OR-A5LD-01,0.0000,5242.16,0.0000,3084.12,675.712,71.9132,1516.960,0.0000,7328.36,667.571,...,0.0,177.7480,0.0,2689.280,419.267,2.7137,165.536,153.3240,1048.8500,647.218
TCGA-OR-A5LE-01,0.0000,11662.90,0.0000,1376.45,3164.360,0.6150,1435.490,0.0000,4175.48,262.620,...,0.0,76.8365,0.0,1541.900,293.372,2.4601,397.929,665.3710,1068.3200,461.892
TCGA-OR-A5LG-01,0.0000,7473.86,0.0000,1879.19,2373.340,14.5603,750.768,0.0000,11884.90,414.506,...,0.0,63.2465,0.0,857.695,312.592,0.0000,227.505,125.5510,230.6900,454.556
TCGA-OR-A5LH-01,0.0000,12384.50,0.4631,1719.48,2238.150,138.9290,1286.480,0.0000,3296.33,415.852,...,0.0,730.2030,0.0,854.415,424.197,0.0000,258.408,141.2450,184.3130,517.280


In [ ]:
y_train

array([ 44.54745701,  55.13364237,  68.74445212,  11.9998685 ,
        88.86477956,  16.1094125 ,  19.03540783,  44.44882796,
        30.31199658,  18.11486997,  57.53361607,  99.8783575 ,
        69.20472104,  66.24584936,  17.78610645,  76.63477661,
        16.1094125 ,  49.215899  ,  67.59377979,  18.47650985,
        39.25436434,  15.25462735,  36.2626163 , 121.2479863 ,
        12.59164283,  29.8188513 ,  66.50886018,  72.39372719,
        31.23253444,  18.14774633,  27.02436138,  33.82976625,
       108.1303219 ,  32.67909393, 113.9165598 ,  35.57221291,
        16.37242332,  49.0843936 ,  24.62438768,  11.3094651 ,
        46.48716179,  95.17703916, 153.6311931 , 127.4944932 ,
        68.25130684,  44.84334418,  12.85465365,   4.10954401,
       152.1517572 ,  31.79143242,  43.29815564,  28.30653911,
        29.09557156,  28.63530263,  23.63809712,  39.58312786,
         5.22733997,  39.35299339,  21.76414505,  52.24052339,
        78.41009962,  36.328369  ])

# Modelling

In [ ]:
import autokeras as ak
from autokeras import StructuredDataRegressor

In [ ]:
model = StructuredDataRegressor(max_trials=5, loss="mean_squared_error")

In [ ]:
model.fit(X_train, y_train, epochs=100)

Trial 5 Complete [00h 05m 32s]
val_loss: 4407.8740234375

Best val_loss So Far: 4407.8740234375
Total elapsed time: 00h 17m 21s
Epoch 1/100
2/2 [==============================] - 104s 258ms/step - loss: 103048720.0000 - mean_squared_error: 103048720.0000
Epoch 2/100
2/2 [==============================] - 0s 259ms/step - loss: 102968352.0000 - mean_squared_error: 102968352.0000
Epoch 3/100
2/2 [==============================] - 0s 250ms/step - loss: 60367260.0000 - mean_squared_error: 60367260.0000
Epoch 4/100
2/2 [==============================] - 0s 257ms/step - loss: 22500600.0000 - mean_squared_error: 22500600.0000
Epoch 5/100
2/2 [==============================] - 1s 251ms/step - loss: 21338418.0000 - mean_squared_error: 21338418.0000
Epoch 6/100
2/2 [==============================] - 0s 249ms/step - loss: 21449434.0000 - mean_squared_error: 21449434.0000
Epoch 7/100
2/2 [==============================] - 0s 254ms/step - loss: 33948036.0000 - mean_squared_error: 33948036.0000
Epoch

In [ ]:
model.evaluate(X_train, y_train)

2/2 [==============================] - 85s 268ms/step - loss: 4031.8547 - mean_squared_error: 4031.8547


[4031.854736328125, 4031.854736328125]

In [ ]:
predicted_y = model.predict(X_test)
# Evaluate the best model with testing data.
print(model.evaluate(X_test, y_test))

1/1 [==============================] - 80s 80s/step - loss: 26011.3633 - mean_squared_error: 26011.3633
[26011.36328125, 26011.36328125]
